# DataFrame III

Los objetivos de aprendizaje son:

1. Valores Duplicados
    + .duplicate()
    + .drop_duplicates()
    + .unique()
    + .nunique()
2. Random Samples
    + Registros
    + Columnas
3. Group by
    + get_group()
    + .agg()
    + Multiples columnas
    


## 1. Valores Duplicados

Pandas cuenta con un grupo de métodos que son bastante útiles para gestionar información duplicada.


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("./Data/pandas/employees.csv")
df.head()

El DataFrame contiene algunos campos que evidentemente son del tipo fecha, ¿Cómo los habrá interpretado la función `read_csv()`?

In [ ]:
df.dtypes

Los ha interpretado como `str`. Podemos indicar a la función qué columnas debe interpretar como fechas.

In [ ]:
df = pd.read_csv("./Data/pandas/employees.csv", parse_dates = ['Start Date', 'Last Login Time'])
df.head()

In [ ]:
df.dtypes

¡Mejor!

In [ ]:
df.sort_values('First Name', inplace = True)
df.head(3)

### duplicated

El método [`.duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) detecta datos duplicados, los  parámetros importantes son:

* subset: Los nombres de las columnas sobre los que verificaremos la condición de duplicado. 
* keep:
    + `"first"`: Marcará como True a los registro duplicados, salvo al primero.
    + `"last"`: Marcará como True a los registro duplicados, salvo a último.
    + `False`: Marcará como True a todas las ocurrencias de duplicados.


In [ ]:
df[['First Name', 'Gender']]

In [ ]:
df[
    df.duplicated(
        subset = ["First Name", "Gender"], keep = False
    )
].head(3)

In [ ]:
df[
    df.duplicated(
        subset = ["First Name", "Gender"], keep = "first"
    )
].head(3)

In [ ]:
df[
    df.duplicated(
        subset = ["First Name", "Gender"], keep = "last"
    )
].head(3)

### drop_duplicates 

Antes de borrar los elementos duplicados en un `DataFrame` es importante ivestigar por qué están ahí.

En cualquier caso, si ya entendimos por qué tenemos duplicados y estamos seguros que queremos borrarlos, podemos usar el método [`.drop_duplicates()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

In [ ]:
df.drop_duplicates(subset = ['First Name', 'Gender'], keep = "first").head()

### unique()

El método `.unique()` nos regresa los posibles valores que existen en una columna de un `DataFrame`.

In [ ]:
df['Gender'].unique()

### nunique()

El método `.nunique()`  aplicado sobre la columna de un `DataFame`, nos regresa cuántos valores únicos existen.

In [ ]:
df['Gender'].nunique()

Podemos configurarlo para que cuente los `NaN`s como otro tipo de valor.

In [ ]:
df['Gender'].nunique(dropna = False)

## 2. Random Samples

Podemos generar muestras aleatorias con el método `.sample()`

In [ ]:
df.sample(frac = .7).head()

In [ ]:
df.sample(frac = .7).head()

In [ ]:
df.sample(frac = .7, random_state = 101).head()

In [ ]:
df.sample(frac = .7, random_state = 101).head()

In [ ]:
df.sample(3, axis = 1).head()

## 3. Group by

Nos ayuda a agregar los datos para poder analizar tendencias.

Para esta sección usaremos un `DataFrame` que contiene información de las 1000 empresas más grandes, según datos de [fortune](https://fortune.com/) correspondientes al 2016.

### .gropuby()



El método [`.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

In [ ]:
fortune = pd.read_csv("./Data/pandas/fortune1000.csv")
fortune.head()

In [ ]:
sector = fortune.groupby(by = ['Sector'])
sector

El método `.groupby()` nos regresa un objeto de la clase `DataFrameGroupby`.

¿Qué es exactamente este objeto? es una especie de extra índice que dice a Pandas dónde mirar cuando más adelante se le pidan hacer cierto tipo de operaciones de agregación.

Esto quedará más claro con un ejemplo de una llamada al atributo `grups`, que nos regresará un diccionario en donde:

* `keys`: Los valores de la variable que usamos en el método`.groupby()`.
* `values`: son los ínices que contienen los registros de cada valor de la variable agrupadora.

In [ ]:
sector.groups

In [ ]:
fortune.loc[[23, 44, 59, 87, 117]]

Si quisiéramos seleccionar todos los registros tal que `Sector == "Aerospace & Defense"`, podríamos hacer lo siguiente:

In [ ]:
fortune.loc[sector.groups['Aerospace & Defense']].head(5)

Pero esto es más directo con el método `.get_group()` 

In [ ]:
sector.get_group('Aerospace & Defense').head(5)

Supongamos que queremos saber en qué sector la media de `Revenue` es más alta. 

In [ ]:
round(sector['Revenue'].mean(),2).sort_values(ascending = False).head()

O quizás queremos saber el percentil 0.95 de la variable `Profits` por Sector. 

In [ ]:
sector['Profits'].quantile(0.95).sort_values(ascending = False).head()

Veamos cómo realizar operaciones de agregación por múltiples columnas manteniendo un formato de `DataFrame`.

In [ ]:
fortune.groupby(
    by = ['Sector', 'Industry']
)['Profits'].max().to_frame().reset_index().head(10)

### agg

El método [`.agg()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), aplicado sobre un `DataFrameGroupBy` nos ofrece una mayo flexibilidad para realizar operaciones de agregación, veamos un ejemplo.

In [ ]:
fortune.groupby(by = ['Sector']).agg(
    mean_profit = ('Profits', lambda x: round(np.mean(x),2)),
    per_95_profit = ('Profits', lambda x: np.percentile(x,95))
    ).reset_index().sort_values('mean_profit', ascending = False).head(10)

### Iteración sobre Grupos

Supongamos que nos interesa saber cuáles son las compañías con el mayor margen de ganancias por sector, no cuál es el máximo margen de ganancias por sector.

In [ ]:
sector = fortune.groupby(by = 'Sector')

In [ ]:
rank2_bysector = []

In [ ]:
for grupo, data in sector:
    rank2_bysector.append(
        data.nlargest(2, "Profits")
    )
pd.concat(rank2_bysector).head(10)